# Assignment 4
#### Based on pre-class work from session 8.1

We consider the eczema medical trial data set again. This time we will compare which of 2 models explain the observed data best.

* Model 1: All studies have the same probability of success.


|Study          | Treatment group |   Control group  |
|---------------|-----------------|------------------|
|Di Rienzo 2014 |     20 / 23     |        9 / 15    |
|Galli 1994     |     10 / 16     |       11 / 18    |
|Kaufman 1974   |     13 / 16     |        4 / 10    |
|Qin 2014       |     35 / 45     |       21 / 39    |
|Sanchez 2012   |     22 / 31     |       12 / 29    |
|Silny 2006     |      7 / 10     |        0 / 10    |
|**Totals**     |    107 / 141    |       57 / 121   |


**Model 1:**

* For each group (treatment and control), all 6 studies have the same fixed, but unknown, probability of success, $\theta_t,\theta_c\in[0,1]$.
* The data follow a binomial distribution in each study, conditioned on the probability of success — $\theta_t$ for treatment or $\theta_c$ for control.
* The priors over $\theta_t$ and $\theta_c$ are uniform.

These assumptions lead to the following model.

* Likelihood: $\prod_{i=1}^6 \text{Binomial}(s_i\,|\,\theta, n_i)$, where $s_i$ is the number of successful recoveries, $f_i$ is the number of failures (did not recover), and $n_i=s_i+f_i$ the number of patients.

* Prior: $\text{Beta}(\theta\,|\,1,1)$ for both $\theta_t$ and $\theta_c$.

* Posterior for treatment group: $\text{Beta}(\theta_t\,|\,108, 35)$.

* Posterior for control group: $\text{Beta}(\theta_c\,|\,58, 65)$.

Since we have closed-form solutions for the posteriors, we can calculate the marginal likelihood by rearranging Bayes' equation: (marginal likelihood) = (likelihood) x (prior) / (posterior).

$$ P(\text{data}) = \left[\prod_{i=1}^6 \text{Binomial}(s_i\,|\,\theta, n_i) \right] \text{Beta}(\theta\,|\,\alpha_0,\beta_0) \,/\, \text{Beta}(\theta\,|\,\alpha_1,\beta_1)$$
where $\alpha_0=1$ and $\beta_0=1$ are the parameters of the prior, and $\alpha_1$ and $\beta_1$ are the parameters of the posterior beta distribution.

Since all factors involving $\theta$ cancel out, we are just left with the normalization constants of the likelihood, the prior and the posterior:

$$\begin{aligned}
P(\text{data})
&= \left[ \prod_{i=1}^6 \left(\begin{array}{c}s_i+f_i \\ s_i\end{array}\right) \right] \frac{\text{B}(\alpha_1,\beta_1)}{\text{B}(\alpha_0,\beta_0)} \\
&= \left[\prod_{i=1}^6 \frac{1}{(s_i+f_i+1)\text{B}(s_i+1,f_i+1)}\right] \frac{\text{B}(\alpha_1,\beta_1)}{\text{B}(\alpha_0,\beta_0)}
\end{aligned}$$

We usually compute the log of the marginal likelihood since the results can vary over many orders of magnitude.

**A word on notation** in the derivation above:

* The beta _distribution_ is written as $\text{Beta}(\theta \,|\, \alpha, \beta)$.
* The beta _function_ is written as $B(\alpha,\beta)$. $B$ is the Greek letter _capital beta_.
* The beta function is part of the normalization constant of the beta distribution.

This is similar to the gamma distribution and the gamma function, where

* the distribution is written as $\text{Gamma}(x \,|\, \alpha, \beta)$,
* the function is written as $\Gamma(\alpha)$,
* the gamma function is part of the normalization constant of the gamma distribution.

**A word on simplifying the expression** in the derivation above:

Just as the gamma function is related to factorials, the beta function is related to combinations:

* $n! = \Gamma(n+1)$ for integer $n$.
* $\binom{n}{k}=\left((n+1)\cdot B(n-k+1, k+1)\right)^{-1}$

The beta function can also be written in terms of the gamma function:

* $B(x,y) = \Gamma(x)\ \Gamma(y)\ /\ \Gamma(x+y)$


## Approach to model testing

1. Define test statistic
2. Draw samples from the posterior and generate posterior predictive distributions of the same size as the control group.
3. For each posterior predictive distribution compute the test statistic.
4. Compute the test statistic of the real data.
5. Compute the percentile of the real data test statistic in the set of replicated data test statistics.
6. Visualize results using histogram
7. Interpret

In [45]:
data_control = np.array([[9, 11, 4, 21, 12, 0], [6, 7, 6, 18, 17, 10]]).T

In [27]:
import scipy.stats as sts
import matplotlib.pyplot as plt

def test_statistic(successes, failures):
    
    n = successes+failures
    
    var = n*successes*failures
    return var

In [28]:
def beta_rvs(alpha, beta, size=1):
    '''
    Generate size samples from the beta distribution. This function
    returns a (size x 1) matrix where each row contains a sample, (p).
    '''
    return sts.beta(alpha, beta).rvs(size)

In [51]:
[y_rep, n_control-y_rep]

[array([61]), array([60])]

In [50]:
#Number of patients in the control group, 12
n_control = np.sum(data_control)

#Storage
posterior_predictive_tests = []

#Draw 1000 samples from the posterior
posterior_samples = beta_rvs(58, 65, size=10000)
for i in range(len(posterior_samples)):
    #For every posterior sample p
    sample_p = posterior_samples[i]
    #Generate a posterior predictive distribution of the same size as control
    y_rep = sts.binom(n_control, sample_p).rvs(1)
    #And compute on it the test statistic
    posterior_predictive_tests.append(test_statistic([y_rep, n_control-y_rep]))

TypeError: list indices must be integers or slices, not tuple

In [43]:
sts.binom(, sample_p).rvs(2)

array([1, 1])

In [ ]:
plt.hist(posterior_predictive_tests)
plt.axvline(x=test_statistic(data), color="r")
plt.title("Posterior predictive check")
plt.ylabel("Frequency (#)")
plt.xlabel("Test statistic")
plt.show()

Resources:

Piech, C. (2016). Beta Distribution. Stanford, CS109.

    Retrieved
    https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/pdfs/22%20Beta.pdf
    
Stattrek. (2019). Binomial Probability Distribution.

    Retrieved
    https://stattrek.com/probability-distributions/binomial.aspx

In [ ]:
def test_statistic(data):
    alpha = np.sum(data[:,0])+1
    beta = np.sum(data[:,1])+1
    
    var = (alpha*beta)/(((alpha+beta)**2) * (alpha + beta + 1))
    return var